# 导包 + 切换路径

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt         # Matplotlib包用于绘图的
plt.rcParams['font.sans-serif'] = ['SimHei'] # 正常显示汉字
plt.rcParams['axes.unicode_minus'] = False   # 正常显示负号

import os

# 手动修改 工作空间目录, 即: 修改相对路径的地址
os.chdir('D:/workspace/ai_20_work_bj/pandasProject/')
os.getcwd()

In [ ]:
# 1. 读取数据源, 获取df对象.
customer_info = pd.read_excel('data/会员信息查询.xlsx')
customer_info.head()

# 1. 按月统计注册的会员数量(即: 月增量)

In [ ]:
# 1. 给上述的 df对象, 新增1列, 充当: 年月字段. 
customer_info.loc[:, '注册年月'] = customer_info['注册时间'].apply(lambda x: x.strftime('%Y-%m'))
customer_info.head()

# 2. 从上述的df对象中, 获取到我们要用的字段. 
customer_info[['会员卡号', '会员等级', '会员来源', '注册时间', '注册年月']]

In [ ]:
# 3. 完成上述的需求, 按月统计注册的会员数量.
month_count = customer_info.groupby('注册年月')[['会员卡号']].count()
month_count.columns = ['月增量']
month_count.head()

In [ ]:
# 4. 对上述的 月增量 进行可视化操作.
# figsize: 设置宽高, color: 设置颜色, legend: 设置图例,  grid: 设置网格线, xlabel: x轴显示内容.  ylabei: y轴显示内容
month_count['月增量'][1:].plot(figsize=(16, 8), color='green', legend=True, grid=True, xlabel='年月', ylabel='月增量')

In [ ]:
# 5. pivot_table()透视表, 也可以实现 groupby()函数的功能, 且: 透视表比groupby()相对更加灵活.
tmp_df = customer_info.pivot_table(index='注册年月', values='会员卡号', aggfunc='count')
tmp_df.columns = ['月增量']
tmp_df

# 2. 按月计算每月的 会员月存量, 

In [ ]:
# 需求2: 按月计算每月的 会员月存量,  月存量 即: 从第1个月 ~ 当前月的 月增量的累加和
month_count.loc[:, '会员存量'] = month_count['月增量'].cumsum()
month_count.head()

In [ ]:
# 增量可视化, secondary_y: 该图表参考 右y轴值
month_count['月增量'][1:].plot(figsize=(16, 8), color='green', legend=True, grid=True, secondary_y=True)
# 存量可视化
month_count['会员存量'].plot(kind='bar', figsize=(16, 8), color='pink', legend=True, grid=True, xlabel='年月', ylabel='月存量')
plt.title('月增量 和 月存量分析结果展示')
plt.show()          # 这行代码可以省略不写. 

# 3. 会员增量等级分布

In [ ]:
# 1. 查看下目前的源数据.
customer_info.head()

In [ ]:
# 2. 使用透视表, 计算: 按月 会员增量等级分布即可.
member_rating = customer_info.pivot_table(index='注册年月', columns='会员等级', values='会员卡号', aggfunc='count')

# 3. 从中过滤出 除了第1个月的数据.
member_rating = member_rating[1:]
member_rating

In [ ]:
# 4. 月增量 会员等级分布结果查看.
# 4.1 构建画布, 坐标系
fig, ax1 = plt.subplots(figsize=(20, 10))
# 4.2 基于ax1, 复制一个新的坐标系 ax2.    ax1: 展示: 白银, 黄金会员.  ax2: 展示 钻石, 铂金会员
ax2 = ax1.twinx()
# 4.3 ax1: 展示: 白银, 黄金会员
member_rating[['白银会员', '黄金会员']].plot(ax=ax1, legend=True, grid=True, color=['blue', 'black'], xlabel='年月', ylabel='白银/黄金')
# 4.4 ax2: 展示 钻石, 铂金会员
member_rating[['钻石会员', '铂金会员']].plot(kind='bar', ax=ax2, legend=True, grid=True, ylabel='钻石/铂金')
# 4.5 设置ax2坐标系的图例到: 左侧
ax2.legend(loc='upper left')        
# 4.6 给图表起个名字
plt.title('月增量会员等级分布', fontsize=20)
# 4.7 绘图
plt.show()          # 可以省略不写. 